# Introduction
This notebook performs the tasks in the exercise

In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
from migrator import Migrator
from database import Database
from environs import Env
import tasks as t
import pandas as pd

## Part 1
Setting up and seeding the database.

In [4]:
env = Env()
env.read_env(".env")

In [5]:
database = Database(
    host=env.str("DB_HOST"),
    port=3307,
    user=env.str("DB_USER"),
    password=env.str("DB_PASSWORD"),
    database=env.str("DB_DATABASE")
)

Connected to: 8.1.0
You are connected to the database: ('mysql',)
-----------------------------------------------



### Create tables

In [ ]:
migrator = Migrator(database, 500)

In [ ]:
migrator.migrate()

### (DANGER) Wipe the DB
🚨 THIS WILL WIPE ALL DATA IN THE TABLES 🚨

In [ ]:
migrator.wipe()

### Seed Database from Data Set

In [ ]:
migrator.seed_users()

In [ ]:
migrator.seed_activities()

In [ ]:
migrator.seed_track_points()

In [ ]:
migrator.create_indices()

## Part 2

In [ ]:
tasks = t.Task(database)

### Task 1

In [ ]:
tasks.task1()

### Task 2

In [ ]:
tasks.task2()

### Task 3

In [ ]:
tasks.task3()

### Task 4

In [ ]:
tasks.task4()

### Task 5

In [ ]:
tasks.task5()

### Task 7

In [ ]:
tasks.task7a()

In [ ]:
tasks.task7b()

### Task 8

In [ ]:
tasks.task8()



In [ ]:
# query = """
#     SELECT DISTINCT left.user_id AS UserID1, right.user_id AS UserID2
#     FROM full AS left
#     INNER JOIN full AS right
#         ON left.user_id != right.user_id
#         AND left.datetime <= right.datetime
#         AND MBRContains(ST_BUFFER(left.geom, 50), right.geom)
#         AND TIME_TO_SEC(TIMEDIFF(right.datetime, left.datetime)) <= 30
# """



query = """
SELECT DISTINCT p1.user_id as UserID1, p2.user_id AS UserID2
FROM full AS p1
INNER JOIN full AS p2
    ON p1.user_id != p2.user_id
LIMIT 10
"""

database.query(query)

In [ ]:
database.cursor.fetchall()


In [ ]:
left_table = """
    CREATE TEMPORARY TABLE p1 AS
        SELECT u.id as user_id, tp.datetime as datetime, tp.geom as geom
        FROM TrackPoints as tp
        INNER JOIN Activities as a
            ON a.id = tp.activity_id
        INNER JOIN Users as u
            on u.id = a.user_id
"""
database.query(left_table)

In [ ]:
right_table = """
    CREATE TEMPORARY TABLE p2 AS
        SELECT u.id as user_id, tp.datetime as datetime, tp.geom as geom
        FROM TrackPoints as tp
        INNER JOIN Activities as a
            ON a.id = tp.activity_id
        INNER JOIN Users as u
            on u.id = a.user_id
"""
database.query(right_table)

In [ ]:
# query = """
# SELECT COUNT(DISTINCT a1.user_id, a2.user_id) as num_users
# FROM Activities AS a1
# JOIN Activities AS a2 ON a1.user_id < a2.user_id
# JOIN TrackPoints AS tp1 ON a1.id = tp1.activity_id
# JOIN TrackPoints AS tp2 ON a2.id = tp2.activity_id AND tp1.id < tp2.id
# WHERE MBRContains(ST_BUFFER(tp1.geom, 50), tp2.geom)
# AND ABS(TIME_TO_SEC(TIMEDIFF(tp1.datetime, tp2.datetime))) <= 30
# AND ST_Distance_Sphere(tp1.geom, tp2.geom) <= 50
# """
# query = """
# WITH user_pairs AS (
#     SELECT a1.user_id AS user_id1, a2.user_id AS user_id2
#     FROM Activities AS a1
#     JOIN Activities AS a2 ON a1.user_id < a2.user_id
#     JOIN TrackPoints AS tp1 ON a1.id = tp1.activity_id
#     JOIN TrackPoints AS tp2 ON a2.id = tp2.activity_id AND tp1.id < tp2.id
#     WHERE MBRContains(ST_BUFFER(tp1.geom, 50), tp2.geom)
#     AND ABS(TIME_TO_SEC(TIMEDIFF(tp1.datetime, tp2.datetime))) <= 30
#     AND ST_Distance_Sphere(tp1.geom, tp2.geom) <= 50
# )
# SELECT COUNT(DISTINCT user_id) as num_users
# FROM (
#     SELECT user_id1 AS user_id FROM user_pairs
#     UNION
#     SELECT user_id2 FROM user_pairs
# ) AS user_ids;
# """
query = """
WITH user_pairs AS (
    SELECT a1.user_id AS user_id1, a2.user_id AS user_id2
    FROM Activities AS a1
    JOIN Activities AS a2 ON a1.user_id < a2.user_id
    JOIN TrackPoints AS tp1 ON a1.id = tp1.activity_id
    JOIN TrackPoints AS tp2 ON a2.id = tp2.activity_id AND tp1.id < tp2.id
    WHERE ST_Distance_Sphere(tp1.geom, tp2.geom) <= 50
    AND ABS(TIME_TO_SEC(TIMEDIFF(tp1.datetime, tp2.datetime))) <= 30
)
SELECT COUNT(DISTINCT user_id) as num_users
FROM (
    SELECT user_id1 AS user_id FROM user_pairs
    UNION
    SELECT user_id2 FROM user_pairs
) AS user_ids;
"""

database.query(query)

In [13]:
query = """
WITH user_pairs AS (
    SELECT DISTINCT a1.user_id AS user_id1, a2.user_id AS user_id2
    FROM Activities a1
    -- Make a combination of all activities
    JOIN Activities a2 ON a1.id < a2.id
        -- We restrict the search space to activities that overlap with a 30 second margin
        -- to limit the number of track point comparisons that we have to perform.
        -- Activities that do not overlap within at least a 30 second margin
        -- should not contain track points that are within 30 seconds of each other.
        AND a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
        AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
        -- Avoid comparing a user to themselves
        AND a1.user_id < a2.user_id
    -- Join in the track points on the two sets of activities
    JOIN TrackPoints p1 ON p1.activity_id = a1.id
    JOIN TrackPoints p2 ON p2.activity_id = a2.id
    -- Then, after restricting the search space, we check for
    -- track points that are close both in time
    WHERE ABS(TIME_TO_SEC(TIMEDIFF(p1.datetime, p2.datetime))) <= 30
    -- and in space
    AND ST_Distance_Sphere(p1.geom, p2.geom) <= 50
)
-- Finally, we select the list of distinct user_ids of users
-- who have been near other users in space and time
SELECT DISTINCT user_id
FROM (
    -- Combine the two columns of user ID pairs into a single column of user IDs
    -- to find the total count of unique users who have been near others
    SELECT user_id1 AS user_id FROM user_pairs
    UNION
    SELECT user_id2 FROM user_pairs
) AS user_ids
-- Order the results by ascending ID
ORDER BY user_id ASC;
"""
database.query(query)

Running statement:
 
WITH user_pairs AS (
    SELECT DISTINCT a1.user_id AS user_id1, a2.user_id AS user_id2
    FROM Activities a1
    -- Make a combination of all activities
    JOIN Activities a2 ON a1.id < a2.id
        -- We restrict the search space to activities that overlap with a 30 second margin
        -- to limit the number of track point comparisons that we have to perform.
        -- Activities that do not overlap within at least a 30 second margin
        -- should not contain track points that are within 30 seconds of each other.
        AND a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
        AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
        -- Avoid comparing a user to themselves
        AND a1.user_id < a2.user_id
    -- Join in the track points on the two sets of activities
    JOIN TrackPoints p1 ON p1.activity_id = a1.id
    JOIN TrackPoints p2 ON p2.activity_id = a2.id
    -- Then, after restricting the search space, we check for


,user_id
0,000
1,001
2,003
3,004
4,005
...,...
116,173
117,174
118,175
119,176


In [45]:
query = """
WITH track_points_of_interest AS (
    SELECT id, activity_id, datetime, geom
    FROM TrackPoints
    WHERE activity_id IN (
        SELECT a1.id AS id
        FROM Activities a1
        WHERE EXISTS (
            SELECT 1
            FROM Activities a2
            WHERE a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
            AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
            AND a1.id < id
            AND a1.user_Id < user_id
        )
    )
)
SELECT COUNT(*)
FROM track_points_of_interest p1
JOIN track_points_of_interest p2 ON p1.id < p2.id
AND p1.activity_id != p2.activity_id
"""
database.query(query)

Running statement:
 
WITH track_points_of_interest AS (
    SELECT id, activity_id, datetime, geom
    FROM TrackPoints
    WHERE activity_id IN (
        SELECT a1.id AS id
        FROM Activities a1
        WHERE EXISTS (
            SELECT 1
            FROM Activities a2
            WHERE a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
            AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
            AND a1.id < id
            AND a1.user_Id < user_id
        )
    )
)
SELECT COUNT(*)
FROM track_points_of_interest p1
JOIN track_points_of_interest p2 ON p1.id < p2.id
AND p1.activity_id != p2.activity_id



In [ ]:
query = """
WITH track_points_of_interest AS (
    SELECT id, activity_id, datetime, geom
    FROM TrackPoints
    WHERE activity_id IN (
        SELECT a1.id AS id
        FROM Activities a1
        WHERE EXISTS (
            SELECT 1
            FROM Activities a2
            WHERE a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
            AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
            AND a1.id < id
            AND a1.user_Id < user_id
        )
    )
)
SELECT COUNT(*)
FROM track_points_of_interest p1
WHERE EXISTS (
    SELECT 1
    FROM track_points_of_interest p2
    WHERE p2.id < p1.id
    AND p2.activity_id != p1.activity_id
    AND ABS(TIME_TO_SEC(TIMEDIFF(p1.datetime, p2.datetime))) <= 30
    -- and in space
    AND ST_Distance_Sphere(p1.geom, p2.geom) <= 50
);
"""
database.query(query)

In [27]:
track_points_of_interest = database.query(
    """
        SELECT tp.id AS tp_id, a.id AS activity_id, a.user_id AS user_id, datetime, ST_Latitude(geom) AS latitude, ST_Longitude(geom) AS longitude
        FROM Activities a
        JOIN TrackPoints tp ON a.id = tp.activity_id
        WHERE a.id IN (
            SELECT a1.id AS id
            FROM Activities a1
            WHERE EXISTS (
                SELECT 1
                FROM Activities a2
                WHERE a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
                AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
                AND a1.id < id
                AND a1.user_Id < user_id
            )
        )
    """
)

Running statement:
 
        SELECT tp.id AS tp_id, a.id AS activity_id, a.user_id AS user_id, datetime, ST_Latitude(geom) AS latitude, ST_Longitude(geom) AS longitude
        FROM Activities a
        JOIN TrackPoints tp ON a.id = tp.activity_id
        WHERE a.id IN (
            SELECT a1.id AS id
            FROM Activities a1
            WHERE EXISTS (
                SELECT 1
                FROM Activities a2
                WHERE a2.start_datetime <= a1.end_datetime + INTERVAL 30 SECOND
                AND a2.end_datetime >= a1.start_datetime - INTERVAL 30 SECOND
                AND a1.id < id
                AND a1.user_Id < user_id
            )
        )
    
Query Finished


In [28]:
track_points_of_interest

,tp_id,activity_id,user_id,datetime,latitude,longitude
0,3021481,000-20081023025304,000,2008-10-23 02:53:04,39.984702,116.318417
1,3021482,000-20081023025304,000,2008-10-23 02:53:10,39.984683,116.318450
2,3021483,000-20081023025304,000,2008-10-23 02:53:15,39.984686,116.318417
3,3021484,000-20081023025304,000,2008-10-23 02:53:20,39.984688,116.318385
4,3021485,000-20081023025304,000,2008-10-23 02:53:25,39.984655,116.318263
...,...,...,...,...,...,...
5927774,9681391,176-20071208013029,176,2007-12-08 03:13:54,39.971000,116.304200
5927775,9681392,176-20071208013029,176,2007-12-08 03:15:01,39.971100,116.304933
5927776,9681393,176-20071208013029,176,2007-12-08 03:15:54,39.970817,116.304250
5927777,9681394,176-20071208013029,176,2007-12-08 03:16:57,39.970867,116.303250


In [19]:
from itertools import combinations
from sklearn.metrics.pairwise import haversine_distances
import numpy as np

comb_rows = combinations(track_points_of_interest.index, 2)

In [23]:
track_points_of_interest["datetime"] = pd.to_datetime(track_points_of_interest["datetime"])
track_points_of_interest[["latitude", "longitude"]] = np.radians(track_points_of_interest[["latitude", "longitude"]])

In [24]:
track_points_of_interest

,id,activity_id,datetime,latitude,longitude
0,3021481,000-20081023025304,2008-10-23 02:53:04,0.697865,2.030139
1,3021482,000-20081023025304,2008-10-23 02:53:10,0.697864,2.030140
2,3021483,000-20081023025304,2008-10-23 02:53:15,0.697864,2.030139
3,3021484,000-20081023025304,2008-10-23 02:53:20,0.697864,2.030139
4,3021485,000-20081023025304,2008-10-23 02:53:25,0.697864,2.030137
...,...,...,...,...,...
5927774,9681391,176-20071208013029,2007-12-08 03:13:54,0.697626,2.029891
5927775,9681392,176-20071208013029,2007-12-08 03:15:01,0.697627,2.029904
5927776,9681393,176-20071208013029,2007-12-08 03:15:54,0.697622,2.029892
5927777,9681394,176-20071208013029,2007-12-08 03:16:57,0.697623,2.029875


In [41]:
first, second = track_points_of_interest.iloc[0], track_points_of_interest.iloc[4]
first, second

(tp_id                      3021481
 activity_id     000-20081023025304
 user_id                        000
 datetime       2008-10-23 02:53:04
 latitude                 39.984702
 longitude               116.318417
 Name: 0, dtype: object,
 tp_id                      3021485
 activity_id     000-20081023025304
 user_id                        000
 datetime       2008-10-23 02:53:25
 latitude                 39.984655
 longitude               116.318263
 Name: 4, dtype: object)

In [61]:
test_df = track_points_of_interest.head(50000)

In [47]:
abs(first["datetime"] - second["datetime"]) < pd.Timedelta(seconds=30)

True

In [14]:
users_close_to_others = set()

In [62]:
for first_idx, second_idx in comb_rows:
    first, second = test_df.iloc[first_idx], track_points_of_interest.iloc[second_idx]
    
    if first.user_id == second.user_id:
        continue

    if first.user_id in users_close_to_others and second.user_id in users_close_to_others:
        continue

    if abs(first["datetime"] - second["datetime"]) > pd.Timedelta(seconds=30):
        continue
    
    distances = haversine_distances([first[["latitude", "longitude"]], second[["latitude", "longitude"]]]) * 6371000
    if distances[0, 1] > 50:
        continue

    users_close_to_others.add(first.user_id)
    users_close_to_others.add(second.user_id)

KeyboardInterrupt: 